In [1]:
import timeit
import os
import pickle
import json
import numpy
from hyperopt import fmin, tpe, hp, Trials
from training import objective_train_model
from predict import eval_train_model
from optparse import OptionParser

In [2]:
def convertlabeltostr(label):
    if label==0:
        return("support")
    elif label==1:
        return("comment")
    elif label==2:
        return("deny")
    elif label==3:
        return("query")
    else:
        print(label)  
        
#%%

### Train and Test the model on the best parameters obtained after tuning and dump the results. 
    . Results contain the entire output of the eval_train_model function
    . Predictions contain the predicted labels of the Test data.

In [3]:
global res
def eval(params):
    global res
    start = timeit.default_timer()
    result = eval_train_model(params)
    # Convert result to scorer.py format
    keys = pickle.loads(result['attachments']['ID'])
    values = pickle.loads(result['attachments']['Predictions'])
    values = [convertlabeltostr(s) for s in values] 
    result_dictionary = dict(zip(keys, values))
    res = result
    out_path = 'output'
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    f = open(os.path.join(out_path,'result.txt'), "w+")
    pickle.dump(result, f)
    f.close()

    with open(os.path.join(out_path,'predictions.txt'), 'w+') as outfile:
        json.dump(result_dictionary, outfile)
    print ("saved result and predictions")
    stop = timeit.default_timer()
    print ("Time: ",stop - start)
    
#%%

In [4]:
def main():
    params_file = 'output/bestparams.txt'
    psearch = False
    if psearch:
        if is_test:
            print '\nStarting parameter search using test parameters...\n'
        else:
            print '\nStarting parameter search...\n'
        params = parameter_search(ntrials, hyperopt_seed, is_test)
        print(params)
        eval(params)
    else:
        with open(params_file, 'rb') as f:
            print '\nLoading best set of model parameters from ', params_file, '...\n'
            params = pickle.load(f)
        print (params)
        eval(params)
        
#%%

main()


Loading best set of model parameters from  output/bestparams.txt ...

{'learn_rate': 0.001, 'num_dense_layers': 2, 'num_lstm_units': 100, 'num_dense_units': 500, 'mb_size': 100, 'num_lstm_layers': 2, 'rng_seed': 364, 'num_epochs': 30, 'l2reg': 0.0}
Retrain model on train+dev set and evaluate on testing set
Epochs: 0/30
Before: target size: 81125
Before: len of targets 3245
Before: len of batchsize 100
Before: Len of inputs: 3245
Before: Len of rmdoublemask: 81125
Before: Size of rmdoublemask: 81125
Len of inputs: 3245
len of targets 3245
saved result and predictions
('Time: ', 133.2505099773407)


In [5]:
res['Params']

{'l2reg': 0.0,
 'learn_rate': 0.001,
 'mb_size': 100,
 'num_dense_layers': 2,
 'num_dense_units': 500,
 'num_epochs': 30,
 'num_lstm_layers': 2,
 'num_lstm_units': 100,
 'rng_seed': 364}

In [6]:
res['status']

'ok'

In [7]:
ids = pickle.loads(res['attachments']['ID'])
ids[0:5]

[u'443938194715713536',
 u'443939406823436288',
 u'443939729071820801',
 u'443940322209329152',
 u'443940443219570689']

In [8]:
pred = pickle.loads(res['attachments']['Predictions'])
pred = [convertlabeltostr(pre) for pre in pred] 
pred[0:5]

['support', 'comment', 'comment', 'comment', 'comment']

In [9]:
print(len(pred))

1066
